In [ ]:
import random
import toml
import pygsheets
from tqdm.auto import tqdm
import Bio.Restriction as Restriction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from itertools import product

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
from paulssonlab.api.util import base_url
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.design as design
import paulssonlab.cloning.primers as primers
import paulssonlab.api.geneious as geneious

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
geneious_sessionmaker = geneious.connect(**config["geneious"])

In [ ]:
reg = registry.Registry(
    gc,
    config["registry"]["folder"],
    geneious_sessionmaker=geneious_sessionmaker,
    geneious_folder="registry",
)

# Config

In [ ]:
olib_oligos = reg[("oLIB", "oligos")]
plib_plasmids = reg[("pLIB", "plasmids")]
plib_maps = reg[("pLIB", "maps")]
flib_fragments = reg[("fLIB", "fragments")]
part_types = reg[("fLIB", "fragments", "Part types")]

In [ ]:
# (reverse, forward)
nonflipped_backbone_primers = ("oLIB175", "oLIB176")
flipped_backbone_primers = ("oLIB177", "oLIB178")

In [ ]:
barcode_uns = "UNS9"

In [ ]:
# enzymes = ("BsaI", "BsmBI")
enzymes = ("BsaI",)

In [ ]:
template = reg.get("pLIB122")["_seq"]

In [ ]:
barcode_overhangs = workflow.overhangs_for(part_types["Eaton_barcode"])

# Barcode-compatible backbone primers

In [ ]:
backbone_primers = {}
for flipped in (False, True):
    for forward, orig_primer_name in zip((False, True), nonflipped_backbone_primers):
        orig_primer_seq = olib_oligos[orig_primer_name]["Sequence"]
        overhang = flib_fragments.find({"Name": barcode_uns})["Sequence"]
        if flipped:
            overhang = sequence.reverse_complement(overhang)
        seq = primers.replace_primer_overhang(
            template,
            orig_primer_seq,
            overhang,
        )
        seq = workflow.normalize_seq_upper(seq)
        name = f"JUMP_{barcode_uns}{'_flipped' if flipped else ''}_{'f' if forward else 'r'}"
        description = f"Same binding region as {orig_primer_name} but with {barcode_uns} overhang{' (flipped)' if flipped else ''}."
        backbone_primers[name] = {
            "Sequence": seq,
            "Description": description,
            "Name": name,
        }

In [ ]:
backbone_primers

In [ ]:
base = {"Author": "Jacob Quinn Shenker", "Date": workflow.date()}

oligo_base = {
    **base,
    "Order date": workflow.date(),
    "Vendor": "IDT",
    "Type": "Primer",
}

apply = {"Sequence": workflow.normalize_seq}
# apply = {"Name": None}

for backbone_primer in backbone_primers.values():
    olib_oligos.upsert(
        {**oligo_base, **backbone_primer},
        apply=apply,
    )

In [ ]:
olib_oligos.local

# Barcode placeholders

In [ ]:
# barcode placeholder: UNS10-ph-UNS9, UNS10-ph_r-UNS9 + normal/flipped UNS10->9 backbone primers
# barcode placeholder: UNS9-ph-UNS1, UNS9-ph_r-UNS1 + normal/flipped UNS1->9 backbone primers

In [ ]:
# generate so random sequences are fixed for each enzyme
placeholders = {
    enzyme_name: design.golden_gate_placeholder(
        getattr(Restriction, enzyme_name), None, *barcode_overhangs, random_flanks=False
    )
    for enzyme_name in enzymes
}

In [ ]:
base = {"Author": "Jacob Quinn Shenker", "Date": workflow.date()}

oligo_base = {
    **base,
    "Order date": workflow.date(),
    "Vendor": "IDT",
    "Type": "IDT Ultramer",
}

part_base = {
    **base,
    "Tags": "3g",
}

# apply = {"Sequence": workflow.normalize_seq}
apply = {"Name": None}
overwrite = False

# for enzyme_name, upstream, antiparallel in product(enzymes, (True,), (True,)):
for enzyme_name, upstream, antiparallel in product(
    enzymes, (True, False), (True, False)
):
    if upstream:
        unses = ("UNS9", "UNS1")
    else:
        unses = ("UNS10", "UNS9")
    placeholder = placeholders[enzyme_name]
    if antiparallel:
        placeholder = sequence.reverse_complement(placeholder)
    seq = (
        flib_fragments.find({"Name": unses[0]})["Sequence"] + placeholder + flib_fragments.find({"Name": unses[1]})["Sequence"]
    )
    name = f"barcode_ph_{enzyme_name}_{'upstream' if upstream else 'downstream'}{'_antiparallel' if antiparallel else ''}"
    description = f"3G part with {unses[0]}/{unses[1]} flanks and {enzyme_name} placeholder{' (reversed)' if antiparallel else ''} for golden gating in Daniel Eaton's barcode library."
    placeholder_oligo = {
        **oligo_base,
        "Description": description,
    }
    oligo_id1 = olib_oligos.upsert(
        {
            **placeholder_oligo,
            "Name": f"{name}_sense",
            "Sequence": workflow.normalize_seq_upper(seq),
        },
        apply=apply,
        overwrite=overwrite
    )
    oligo_id2 = olib_oligos.upsert(
        {
            **placeholder_oligo,
            "Name": f"{name}_antisense",
            "Sequence": workflow.normalize_seq_upper(sequence.reverse_complement(seq)),
        },
        apply=apply,
        overwrite=overwrite
    )
    usage = f"{oligo_id1}={oligo_id2}"
    part_row = {
        **part_base,
        "Name": name,
        "Description": description,
        "Sequence": workflow.normalize_seq(seq),
        "Usage": usage,
        "Type": "",
        "Upstream overhang": "",
        "Downstream overhang": "",
        "Species/codon usage": "E. coli",
    }
    flib_fragments.upsert(part_row, apply=apply, overwrite=overwrite)

In [ ]:
olib_oligos.local

In [ ]:
flib_fragments.local

In [ ]:
olib_oligos.commit()
flib_fragments.commit()

# Validation

In [ ]:
template

In [ ]:
primers_

In [ ]:
olib_oligos["oLIB176"]["Sequence"]

In [ ]:
# flipped_, upstream_, ph_name = True, True, "barcode_ph_BsaI_upstream_antiparallel"
flipped_, upstream_, ph_name = True, True, "barcode_ph_BsaI_upstream_antiparallel"

# TODO: use other primers for downstream

ph = flib_fragments.find("Name": ph_name)["Sequence"]
if flipped_:
    primers_ = (
        olib_oligos["oLIB177"]["Sequence"],
        backbone_primers["JUMP_UNS9_flipped_f"]["Sequence"],
    )
    nonbarcoded_primers = (
        olib_oligos["oLIB177"]["Sequence"],
        olib_oligos["oLIB178"]["Sequence"],
    )
else:
    primers_ = (
        backbone_primers["JUMP_UNS9_r"]["Sequence"],
        olib_oligos["oLIB176"]["Sequence"],
    )
    nonbarcoded_primers = (
        olib_oligos["oLIB175"]["Sequence"],
        olib_oligos["oLIB176"]["Sequence"],
    )
backbone_product = sequence.pcr(template, *primers_)
nonbarcoded_backbone_product = sequence.pcr(template, *nonbarcoded_primers)
dummy_insert = (
    flib_fragments.find("Name": "UNS1")["Sequence"]
    + "A" * 30
    + "G" * 5
    + "A" * 10
    + flib_fragments.find("Name": "UNS10")["Sequence"]
)
if upstream_:
    frags = [ph, dummy_insert, backbone_product]
else:
    frags = [dummy_insert, ph, backbone_product]
barcoded_product = sequence.assemble(frags, method="gibson")
nonbarcoded_product = sequence.assemble(
    [dummy_insert, nonbarcoded_backbone_product], method="gibson"
)

In [ ]:
print(barcoded_product.seq)